# Abbaspour - 610398147 - Clustering MNIST
The problem is reduce feature of MNIST dataset images with an autoencoder becomes tuned and trained, and use encoded images (from three set train set, test set and validation set (35% of test set)) to get classified through a neural network classifier.
Firrst some methods of some libraries are imported for calculations and plottings.

In [ ]:
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from keras.utils import to_categorical
from matplotlib.pyplot import figure, ion, legend, plot, savefig, scatter, show, subplot, subplots, title, xlabel, ylabel
from numpy import argmax, argsort, random, reshape, sqrt
from os import environ
from pandas import read_csv
from seaborn import histplot
# from scipy.linalg import eigh
# from scikeras.wrappers import KerasClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from warnings import filterwarnings
filterwarnings("ignore")
environ["KMP_DUPLICATE_LIB_OK"], environ["TF_CPP_MIN_LOG_LEVEL"], environ["TF_ENABLE_ONEDNN_OPTS"] = "TRUE", "3", "0"

# Loading Dataset & Defining Variables (+ Answering Questions)
Train set and test set are read as data frames. Accoring to histogram plot located in 'Evaluations' folder, whole dataset is roughly balanced and dataset doesn't seem to be resampled.
As a preprocessing, values are divided by 255 as they're in range (0, 255) as a normalization.
The reason of this action is to avoid the possibility of exploding gradients due to the high range the pixels and improve the convergence speed.

In [ ]:
# Loading & normalizing the dataset
trainSet, testSet = read_csv("mnist_train.csv"), read_csv("mnist_test.csv")
df = trainSet._append(testSet, ignore_index = True)
X_train, X_test, y_train, y_test, Optimizers, Activations, Initializers, Architecture, learningRates, lossFunction, Accuracy, Loss, validationAccuracy, validationLoss, Title, Alpha, localPopulation, Numbers, numberIterations, numberGenerations, Fitness, Parameters, Tuner = trainSet.drop(["label"], axis = 1).astype("float32") / 255., testSet.drop(["label"], axis = 1).astype("float32") / 255., trainSet["label"].astype("int"), testSet["label"].astype("int"), {Adam: "Adam" , Nadam: "Nadam", SGD: "SGD"}, ["linear", "relu", "tanh"], ["glorot_normal", "glorot_uniform", "zero"], (500, 100, 30), [0.001, 0.01, 0.1, 0.2, 0.25, 0.3, 0.5, 0.8], "binary_crossentropy", "accuracy", "loss", "val_accuracy", "val_loss", "Autoencoder Evaluation", 0.6, 1, 10, 1, 1, [], [], {}
dfX_train, optimizerNames, localBound, number_test_samples, encoder_input_dimension, decoder_input_dimension, number_hidden_layers, Beta = X_train, list(Optimizers.values()), X_train.shape[1], len(X_test), Architecture[0], Architecture[-1], len(Architecture) - 1, 1 - Alpha

# Defining the encoder input & decoder input dimentions.
Image, Length, Start = Input(shape = (localBound,)), int(sqrt(localBound)), int(number_test_samples * 0.35)

# 0.35% of test samples are chosen as validation samples
validationSet = testSet.iloc[Start : number_test_samples - Start]
y_val, X_val, arrtestSet = validationSet["label"], validationSet.drop(["label"], axis = 1), testSet.to_numpy()
print(f"There're {len(X_train)} samples of {Length} X {Length} images as train samples and {number_test_samples} samples as test samples.")

# Plot samples
for Number in range(Numbers):
    fig = figure
    imshow(arrX_train[Number], cmap = "gray")
    savefig(dpi = 1200)
    show()

# Plotting raw data records
Figure, ax = subplots()
ax.pie(df["label"].value_counts(), labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], autopct = '%1.1f%%', shadow = True, startangle = 90)
ax.axis("equal")
savefig(dpi = 1200)
show()
histplot(df["label"])
savefig("MNIST Histogram", dpi = 1200)
show()

# Tuning Autoencoder
Target is to minimizing binary crossentropy value as error function.
Autoencoder is fine tuned by searching step by step between three optimizers 'Adam', 'Nadam' & 'SGD', weight initializers methods 'Glorot Normal', 'Glorot Uniform' and 'Zero' and activation functions 'Linear', 'Relu' and 'Tanh'.
Architecture is (500, 100, 30) for hidden layer. Size of input and output of target autoencoder equals 784 because the program has to use and process all 28 x 28 = 784 pixels of each image of the dataset.
When best optimizer is found by searching, model would be search for good learning rate between candidate learning rates.
In tuning, batch size equals to 500 and search for each tuple of hyperparameters comes done in three epochs as a good group of hyperparameter choices.

In [ ]:
# Fine tuning hyperparameters
print("\nTuning autoencoder...")
for Optimizer in optimizerNames:
    for Initializer in Initializers:
        for Activation in Activations:
            # Define & build the encoder & decoder
            decoderInput  = Input(shape = (decoder_input_dimension,))
            Encoded, Decoded = Dense(encoder_input_dimension, activation = Activation, kernel_initializer = Initializer)(Image), Dense(Architecture[-2], activation = Activation, kernel_initializer = Initializer)(decoderInput)
            for hiddenLayer in range(1, number_hidden_layers):
                Encoded, Decoded = Dense(Architecture[hiddenLayer], activation = Activation, kernel_initializer = Initializer)(Encoded), Dense(Architecture[-(hiddenLayer + 2)], activation = Activation, kernel_initializer = Initializer)(Decoded)

            Encoder, Decoder = Model(Image, Dense(decoder_input_dimension, activation = Activation)(Dense(Architecture[-2], activation = Activation)(Encoded)), name = "Encoder"), Model(decoderInput, Dense(localBound, activation = "sigmoid", kernel_initializer = Initializer)(Decoded), name = "Decoder")
                
            # Build main autoencoder
            Autoencoder = Model(Image, Decoder(Encoder(Image)))

            # Compile the autoencoder
            Autoencoder.compile(loss = lossFunction, metrics = ["accuracy"], optimizer = Optimizer)

            # Fit the data on the autoencoder
            Autoencoder = Autoencoder.fit(X_train, X_train, batch_size = 500, epochs = 3, shuffle = True, validation_data = (X_val, X_val), verbose = 0)
            Results = Autoencoder.history
            Tuner[(Optimizer, Initializer, Activation)] = Alpha * (Results[validationAccuracy][0] - Results[validationLoss][0]) + Beta * (Results[Accuracy][0] - Results[Loss][0])

Optimizer, Initializer, Activation = max(Tuner, key = Tuner.get)
optimizerName = Optimizer
Optimizer, decoderInput, Tuner = list(Optimizers.keys())[optimizerNames.index(optimizerName)], Input(shape = (decoder_input_dimension,)), {}

# Define & build well-tuned encoder & decoder
Encoded, Decoded = Dense(encoder_input_dimension, activation = Activation, kernel_initializer = Initializer)(Image), Dense(Architecture[-2], activation = Activation, kernel_initializer = Initializer)(decoderInput)
for hiddenLayer in range(1, number_hidden_layers):
    Encoded, Decoded = Dense(Architecture[hiddenLayer], activation = Activation, kernel_initializer = Initializer)(Encoded), Dense(Architecture[-(hiddenLayer + 2)], activation = Activation, kernel_initializer = Initializer)(Decoded)

Encoder, Decoder = Model(Image, Dense(Architecture[-1], activation = Activation)(Dense(Architecture[-2], activation = Activation)(Encoded)), name = "Encoder"), Model(decoderInput, Dense(localBound, activation = "sigmoid", kernel_initializer = Initializer)(Decoded), name = "Decoder")

# Search for good-enough learning rate
for learningRate in learningRates:
    # Build main autoencoder
    Autoencoder = Model(Image, Decoder(Encoder(Image)))

    # Compile the autoencoder
    Autoencoder.compile(loss = lossFunction, metrics = ["accuracy"], optimizer = Optimizer(learning_rate = learningRate))

    # Fit the data on the autoencoder
    Autoencoder = Autoencoder.fit(X_train, X_train, batch_size = 500, epochs = 3, shuffle = True, validation_data = (X_val, X_val), verbose = 0)
    Results = Autoencoder.history
    Tuner[learningRate] = Alpha * (Results[validationAccuracy][0] - Results[validationLoss][0]) + Beta * (Results[Accuracy][0] - Results[Loss][0])

learningRate = max(Tuner, key = Tuner.get)
print(f"\n\nOptimal optimizer, primary weights initialization method, activation functions for hidden layers and learning rate for autoencoder neural network are {optimizerName}, {Initializer}, {Activation} and {learningRate}, respectively.\n\nTraining optimal autoencoder...")

# Training Autoencoder
Autoencoder is now constructed with chosen values of hyperparameters.

In [ ]:
# Build main autoencoder
Autoencoder = Model(Image, Decoder(Encoder(Image)))

# Compile the autoencoder
Autoencoder.compile(loss = lossFunction, metrics = ["accuracy"], optimizer = Optimizer(learning_rate = learningRate))
print(Autoencoder.summary())

# Saving model
Autoencoder.save("Autoencoder.keras")

# Fit the data on the autoencoder
# Test set is now the validation set of model!
autoencoderHistory = Autoencoder.fit(X_train, X_train, batch_size = 500, epochs = 15, shuffle = True, validation_data = (X_test, X_test), verbose = 0)

# Autoencoder Evaluation Plot
Final loss value and accuracy of model's perfomance on train set and test set is plotted.

In [ ]:
# Plot the training history (losses)
figure(figsize = (13, 5))
ion()
plot(autoencoderHistory.history[Loss])
plot(autoencoderHistory.history[validationLoss])
title(Title)
ylabel("Loss")
xlabel("Epochs")
legend(["Train Loss", "Test Loss"], loc = "upper right")
savefig(Title, dpi = 1200)
show()

# Plot the training history (accuracies)
figure(figsize = (13, 5))
ion()
plot(autoencoderHistory.history[Accuracy])
plot(autoencoderHistory.history[validationAccuracy])
title(Title)
ylabel("Accuracy")
xlabel("Epochs")
legend(["Train Accuracy", "Test Accuracy"], loc = "upper right")
savefig(Title, dpi = 1200)
show()

# Datasets Encoding
Datasets get encoded in their corresponding batch sizes (small enough to get better accuracies and also save time!) which are 60000, 1000 and 1000 for train set, test set and validation set, respectively.

In [ ]:
# Encoding images
print("\nEncoding images...")
X_train, X_test, X_val, lossFunction, Title, Tuner = Encoder.predict(X_train, batch_size = 60000, verbose = 0), Encoder.predict(X_test, batch_size = 1000, verbose = 0), Encoder.predict(X_val, batch_size = 1000, verbose = 0), "sparse_categorical_crossentropy", "Classifier Evaluation", {}

# Classifier Construction & Training
A classifier with two hidden layers contain 'ReLU' activation function, and a output layer with 'softmax' activation function; because this function assign decimal probabilities to each class in this multi-class problem. These decimal probabilities must add up to 1. This additional constraint helps training converge more quickly than it otherwise would. 
Loss function was Sparse Categorical Crossentropy during training this classifier.

In [ ]:
print("\n\nTraining the classifier...\n")
# Defining the model
Classifier = Sequential()
Classifier.add(Dense(512, input_dim = 30, kernel_initializer = "uniform", activation = "relu"))
Classifier.add(Dense(256, kernel_initializer = "uniform", activation = "relu"))
Classifier.add(Dense(10, kernel_initializer = "uniform", activation  = "softmax"))

# Compiling the model
Classifier.compile(loss = lossFunction, metrics = ["accuracy"], optimizer = "adam")#Optimizer(learning_rate = learningRate))

print(Classifier.summary())

# Training the model
classifierHistory = Classifier.fit(X_train, y_train, batch_size = 60000, epochs = 30, verbose = 0, validation_data = (X_test, y_test))


# Neural Network based Classifier Evaluation Plot
Final loss value and accuracy of model's perfomance on train set and test set is plotted.

In [ ]:
# Plotting the training history
figure(figsize = (13, 5))
ion()
plot(classifierHistory.history[Loss])
plot(classifierHistory.history[validationLoss])
title(Title)
ylabel("Loss")
xlabel("Epochs")
legend(["Train Loss", "Test Loss"], loc = "upper right")
savefig(Title, dpi = 1200)
show()

# Plot the training history (accuracies)
figure(figsize = (13, 5))
ion()
plot(classifierHistory.history[Accuracy])
plot(classifierHistory.history[validationAccuracy])
title(Title)
ylabel("Accuracy")
xlabel("Epochs")
legend(["Train Accuracy", "Test Accuracy"], loc = "upper right")
savefig(Title, dpi = 1200)
show()

# Results
According to plots in 'Evaluations' folder, training the tuned autoencoder in 15 epochs makes good accuracy enough.
Batch sizes and epochs for both training the autoencoder, classifier and tuning autoencoder was chosen by hand.
Tuning classifier would decrease accuracy so it became commented.
After testing whole algorithm for times, accuracy settled in interval (4, 6) (two accuracies captured as samples of results are located in 'Evaluations' folder.)

In [ ]:
# , Accuracy
Title = "Confusion Matrix"
print(f"\nEvaluating classifier...\n\nModel's score: {Classifier.evaluate(X_test, y_test, batch_size = 1000, verbose = 0)}")
ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, argmax(Classifier.predict(X_test, batch_size = 1000, verbose = 0), axis = 1))).plot()
title(Title)
savefig(Title, dpi = 1200)
show()

# Challenges
Main challenge was to fine tune autoencoder automatically (especially for different architectures) sync encoded images for getting classified with the neural network base classifier. different clustering algorithms!